In [1]:
import pandas as pd
from sklearn.metrics import confusion_matrix
from sklearn.cross_validation import train_test_split
from sklearn.tree import DecisionTreeClassifier , DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import BayesianRidge,LinearRegression,SGDClassifier
from sklearn.neighbors.nearest_centroid import NearestCentroid

/opt/conda/lib/python3.6/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
def acuaracy(cm):
    diagonal_index= 0
    total=0
    diagonal_total=0
    for x in cm:
        index=0
        for y in x:
            if(diagonal_index==index):
                diagonal_total=diagonal_total+y
            total=total+y
            index=index+1
        diagonal_index=diagonal_index+1
    acuaracy =  diagonal_total/total
    return acuaracy

In [8]:
def boostrap(modelos,modelo_strg,maxima_iteracion,tamanio_muestra):
    Datos=[]
    string=["modelo","promedio","minimo","maximo"]
    Datos.append(string)
    index=0;
    for clf in modelos:
        minimo =1;
        maximo=-1;
        promedio=0;
        temporal=[]
        for x in range(0,maxima_iteracion):
            pred_train, pred_test, tar_train, tar_test = train_test_split(predictors, targets, test_size=tamanio_muestra)
            temporal_y = tar_train.values
            temporal_x = pred_train.values
            clf = clf.fit(temporal_x, temporal_y)
            y_true = tar_test.values
            y_pred = clf.predict(pred_test)
            try:
                cnf_matrix = confusion_matrix(y_true, y_pred, labels=labels)
            except:
                y_pred2=[]
                for x in y_pred:
                    y_pred2.append(round(x))
                cnf_matrix = confusion_matrix(y_true, y_pred2, labels=labels)
            dato= acuaracy(cnf_matrix)
            promedio=promedio+dato
            if(dato<minimo):
                minimo=dato
            if(dato>maximo):
                maximo=dato
        temporal.append(modelo_strg[index])        
        temporal.append(promedio/maxima_iteracion)
        temporal.append(minimo)
        temporal.append(maximo)
        Datos.append(temporal)
        index=index+1
    return Datos 

<hr>

In [4]:
f = open('../final.csv','r')
final = pd.read_csv(f)
final.drop('Unnamed: 0', axis=1, inplace=True)
final.drop('Sales', axis=1, inplace=True)

In [5]:
predictors = final[["StoreType","Assortment","CompetitionDistance_int","Customers","Open_int"]]
targets = final.Sales_rango 
labels=[0,1,2,3]

<hr>

In [9]:
#modelos
AC = DecisionTreeClassifier(max_depth=5) # 
AR = DecisionTreeRegressor(max_depth=5)
RF = RandomForestClassifier(n_estimators=10,max_depth=5)
RN = MLPClassifier(hidden_layer_sizes=(5,), max_iter=10)
GN = GaussianNB()
LN = LinearRegression()
BR = BayesianRidge()
SGD= SGDClassifier(loss="hinge", penalty="l2")
NC = NearestCentroid()
modelos=[AC,AR,RF,RN,GN,LN,BR,SGD,NC]
modelos_string=["AC","AR","RF","RN","GN","LN","BR","SGD","NC"]

In [10]:
boostrap(modelos,modelos_string,10,.1)

/opt/conda/lib/python3.6/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


[['modelo', 'promedio', 'minimo', 'maximo'],
 ['AC', 0.72558960293351416, 0.72412776122924472, 0.72798143942745352],
 ['AR', 0.83536044671208509, 0.83365283471456242, 0.83694615664415406],
 ['RF', 0.71763057775680539, 0.70730724235900155, 0.72766685345208959],
 ['RN', 0.64048524886699898, 0.41582367456080849, 0.73195308736642384],
 ['GN', 0.65159603228438578, 0.64969868562047173, 0.65401441196999632],
 ['LN', 0.59807963628947536, 0.59510357815442561, 0.60167538427623801],
 ['BR', 0.59793858903211128, 0.59536177921908939, 0.60169533981944434],
 ['SGD', 0.40463227848723465, 0.27214636112503809, 0.47803305118903666],
 ['NC', 0.38276363779357264, 0.34967214242880035, 0.41636436920596531]]